In [1]:
%pip install --upgrade --quiet pymilvus[model] langchain-milvus langchain-openai
%pip install -qU langchain_community pypdf

zsh:1: no matches found: pymilvus[model]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_ollama import OllamaLLM

In [3]:
CONNECTION_URI = "http://localhost:19530"

In [4]:
ollama_embedding_func = OllamaLLM(model="moondream")

In [ ]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Setup
def setup_ollama_pdf_vectorstore(file_path, embedding_model="moondream"):
    try:
        # 1. Load PDF
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()

        # 2. Text Splitting (Optional but recommended)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        splits = text_splitter.split_documents(docs)

        # 3. Ollama Embeddings
        embeddings = OllamaEmbeddings(
            model=embedding_model,  # You can change this to other Ollama models
            num_gpu=1  # Optional: specify GPU usage
        )

        # 4. Create Chroma Vector Store
        vector_store = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            collection_name="ollama_pdf_collection"
        )

        return vector_store

    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None

files = os.listdir("papers")
print(files)

for file in files:
    vector_store = setup_ollama_pdf_vectorstore(f"papers/{file}")
    print(f"Vector store for {file} created: {vector_store}")
    print("")

['2303.05510v1.pdf', '2203.07814v1.pdf']


In [ ]:
# Perform Similarity Search
contexts = ""
query = input("Enter your query: ")

if vector_store:
    results = vector_store.similarity_search(query, k=10)
    
    for doc in results:
        print("\n--- Relevant Document Excerpt ---")
        print(doc.page_content)
        contexts = contexts + doc.page_content

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize Ollama Chat Model
chat_model = ChatOllama(
    model="llama3.2:1b",  # You can change to other models like mistral, phi3
    temperature=0.7,  # Creativity level
    num_gpu=1  # Optional GPU configuration
)

# Simple Chat Interaction
messages = [
    SystemMessage(content="You are a helpful assistant for my research."),
    HumanMessage(content= f"Query + {query} with the context + {contexts}")
]

# Generate Response
response = chat_model.invoke(messages)
print("--- Ollama Response ---")
print(response.content)

ConnectError: [Errno 61] Connection refused